In [ ]:
import numpy as np

def read_poscar(poscar_file):
    """Read POSCAR file and return header info and atomic coordinates"""
    with open(poscar_file, 'r') as f:
        lines = f.readlines()
    
    # Read header information
    comment = lines[0].strip()
    scaling_factor = float(lines[1])
    lattice_vectors = [list(map(float, line.split())) for line in lines[2:5]]
    atom_types = lines[5].split()
    atom_counts = list(map(int, lines[6].split()))
    
    # Determine coordinate type (Direct or Cartesian)
    coord_type = lines[7].strip().lower()[0] if len(lines) > 7 else 'd'
    
    # Read atomic coordinates
    coords = []
    start_line = 8
    for count in atom_counts:
        for i in range(count):
            coords.append(list(map(float, lines[start_line + i].split()[:3])))
        start_line += count
    
    return {
        'comment': comment,
        'scaling_factor': scaling_factor,
        'lattice_vectors': np.array(lattice_vectors),
        'atom_types': atom_types,
        'atom_counts': atom_counts,
        'coord_type': coord_type,
        'coordinates': np.array(coords)
    }

def write_poscar(data, output_file):
    """Write POSCAR file from data dictionary"""
    with open(output_file, 'w') as f:
        f.write(data['comment'] + '\n')
        f.write(f"  {data['scaling_factor']:.16f}\n")
        for vec in data['lattice_vectors']:
            f.write("  {:18.16f}  {:18.16f}  {:18.16f}\n".format(*vec))
        f.write("  " + "  ".join(data['atom_types']) + "\n")
        f.write("  " + "  ".join(map(str, data['atom_counts'])) + "\n")
        f.write(data['coord_type'].capitalize() + "\n")
        for coord in data['coordinates']:
            f.write("  {:18.16f}  {:18.16f}  {:18.16f}\n".format(*coord))

def c2x_rotation(poscar_data):
    """Perform C2x rotation (180° about x-axis) on atomic coordinates"""
    # Create C2x rotation matrix (180° about x-axis)
    c2x_matrix = np.array([
        [1,  0,  0],
        [0, -1,  0],
        [0,  0, -1]
    ])
    
    # Apply rotation to coordinates
    if poscar_data['coord_type'] == 'd':  # Direct coordinates
        # For fractional coordinates, we need to rotate the lattice vectors first
        new_lattice = c2x_matrix @ poscar_data['lattice_vectors']
        poscar_data['lattice_vectors'] = new_lattice
        # Then rotate the atomic positions (same matrix for fractional coords)
        poscar_data['coordinates'] = np.dot(poscar_data['coordinates'], c2x_matrix.T)
    else:  # Cartesian coordinates
        poscar_data['coordinates'] = np.dot(poscar_data['coordinates'], c2x_matrix.T)
    
    return poscar_data

def main():
    input_file = 'POSCAR'  # Input POSCAR file
    output_file = 'POSCAR_C2x'  # Output POSCAR file after rotation
    
    # Read original POSCAR
    poscar_data = read_poscar(input_file)
    
    # Perform C2x rotation
    rotated_data = c2x_rotation(poscar_data)
    
    # Write rotated POSCAR
    write_poscar(rotated_data, output_file)
    print(f"C2x rotation applied and saved to {output_file}")

if __name__ == '__main__':
    main()